# Silver

## Criar schema

In [0]:
create schema if not exists juntos_somos_mais.silver

## Explorando os dados

In [0]:
select * from juntos_somos_mais.bronze.transactions_raw
limit 10

TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
560773,7/20/2019,22179,Set 10 Night Owl Lights,16.18,1,14773.0,United Kingdom
560773,7/20/2019,22182,Cake Stand Victorian Filigree Small,14.48,3,14773.0,United Kingdom
560773,7/20/2019,22190,Local Cafe Mug,12.77,1,14773.0,United Kingdom
560773,7/20/2019,22196,Small Heart Measuring Spoons,11.92,1,14773.0,United Kingdom
560773,7/20/2019,22199,Frying Pan Red Retrospot,14.48,2,14773.0,United Kingdom
560773,7/20/2019,22203,Milk Pan Red Retrospot,13.62,2,14773.0,United Kingdom
560773,7/20/2019,22208,Wood Stamp Set Thank You,11.1,7,14773.0,United Kingdom
560773,7/20/2019,22209,Wood Stamp Set Happy Birthday,11.1,2,14773.0,United Kingdom
560773,7/20/2019,22210,Wood Stamp Set Best Wishes,11.1,5,14773.0,United Kingdom
560773,7/20/2019,22211,Wood Stamp Set Flowers,11.1,5,14773.0,United Kingdom


In [0]:
-- Identificando os valores nulos
select 
  sum(if(TransactionNo is null, 1,0)) null_TransactionNo, 
  sum(if(Date is null, 1,0)) null_Date, 
  sum(if(ProductNo is null, 1,0)) null_ProductNo, 
  sum(if(ProductName is null, 1,0)) null_ProductName, 
  sum(if(Price is null, 1,0)) null_Price, 
  sum(if(Quantity is null, 1,0)) null_Quantity, 
  sum(if(CustomerNo is null,1,0)) null_CustomerNo, 
  sum(if(Country is null,1,0)) null_Country
from juntos_somos_mais.bronze.transactions_raw

null_TransactionNo,null_Date,null_ProductNo,null_ProductName,null_Price,null_Quantity,null_CustomerNo,null_Country
0,0,0,0,0,0,55,0


In [0]:
-- Identificando valores duplicados
select 
  TransactionNo,
  Date,
  ProductNo,
  ProductName,
  Price,
  Quantity,
  CustomerNo,
  Country, 
  count(*)
from juntos_somos_mais.bronze.transactions_raw
group by 
  TransactionNo,
  Date,
  ProductNo,
  ProductName,
  Price,
  Quantity,
  CustomerNo,
  Country
having count(*) > 1
order by count(*) desc
limit 10

TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,count(*)
555524,6/5/2019,22698,Pink Regency Teacup And Saucer,13.27,1,16923.0,United Kingdom,20
555524,6/5/2019,22697,Green Regency Teacup And Saucer,13.27,1,16923.0,United Kingdom,12
547651,3/24/2019,82613A,Metal Sign Cupcake Single Hook,11.53,1,16904.0,United Kingdom,8
572861,10/26/2019,22775,Purple Drawerknob Acrylic Edwardian,11.53,12,14102.0,United Kingdom,8
538514,12/12/2018,21756,Bath Building Block Word,16.35,1,15044.0,United Kingdom,6
541266,1/16/2019,21755,Love Building Block Word,16.35,1,15673.0,United Kingdom,6
541266,1/16/2019,21754,Home Building Block Word,16.35,1,15673.0,United Kingdom,6
540524,1/9/2019,21756,Bath Building Block Word,16.35,1,16735.0,United Kingdom,6
547712,3/24/2019,22699,Roses Regency Teacup And Saucer,13.27,1,16905.0,United Kingdom,5
574481,11/4/2019,23084,Rabbit Night Light,12.38,1,18022.0,United Kingdom,5


### Pontos de atenção


Na camada **Bronze**, que armazena os dados exatamente como recebidos da fonte, foram identificadas inconsistências que impactam a qualidade e a modelagem dos dados downstream:

- A coluna Date está sendo ingerida como string, quando o tipo ideal seria date, comprometendo a manipulação temporal e a eficiência em queries;
- Existem registros duplicados, onde se espera unicidade por item de transação — o que pode levar a contagens incorretas e distorções em métricas agregadas;
- A coluna CustomerNo apresenta nulls, embora esse campo deva ser obrigatório para rastreabilidade do cliente;
- O tipo da coluna CustomerNo está definido como float, quando deveria ser bigint, o que pode causar perda de precisão e problemas de junção com outras tabelas relacionais;
- Não há um identificador explícito que diferencie o tipo de transação por item de produto, o que pode dificultar identificação de alterações nos registros.

## Criar tabela

In [0]:
-- Criar TEMP VIEW com limpeza e transformação
create or replace temp view new_transactions as
select 
distinct
  cast(replace(TransactionNo, 'C', '') as int) transaction_id,
  to_date(Date, 'M/d/yyyy') date,
  ProductNo product_id,
  ProductName product_name,
  Price price, 
  abs(Quantity) quantity,
  CustomerNo customer_id,
  Country customer_country, 
  if(TransactionNo like 'C%', 'cancelled', 'not cancelled') transaction_status
from juntos_somos_mais.bronze.transactions_raw

In [0]:
create table if not exists juntos_somos_mais.silver.transactions_cleaned (
  transaction_id int,
  date date,
  product_id string,
  product_name string,
  price double,
  quantity int,
  customer_id bigint,
  customer_country string, 
  transaction_status string
)
using delta
comment "Cleaned data from transactions: no duplicates and transformed columns" 

In [0]:
merge into juntos_somos_mais.silver.transactions_cleaned as target
using new_transactions as source
on target.transaction_id = source.transaction_id
and target.product_id = source.product_id 
and target.date = source.date
-- Caso o pedido seja cancelado
when matched then
  update set
    target.transaction_status = source.transaction_status
-- Novos registros
when not matched then
  insert (
    transaction_id,
    date,
    product_id,
    product_name,
    price,
    quantity,
    customer_id,
    country_customer,
    transaction_status
  )
  values (
    source.transaction_id,
    source.date,
    source.product_id,
    source.product_name,
    source.price,
    source.quantity,
    source.customer_id,
    source.country_customer,
    source.transaction_status
  )

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
531150,0,0,531150
